In [27]:
# Instalasi dan import library
!pip install Sastrawi --quiet

import os
import re
import csv
import string
import nltk
import pandas as pd
import requests
import matplotlib.pyplot as plt
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split

# Download resource NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Aktifkan progress bar pada pandas
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [40]:
df = pd.read_csv('https://raw.githubusercontent.com/MIAbidin/Sentimen-Timnas-App/refs/heads/main/Notebook/dataset/dataset_raw.csv')
df

,text
0,NaN
1,NaN
2,Topp ❤️
3,"Alhamdulillah... Tidak apa-apa, yang penting b..."
4,Doa terbaik untuk timnas Indonesia. Insya Alla...
...,...
21040,Semoga lawan Jepang pemain tetap rendah diri.....
21041,Gazzzz Garudaakkuuuu
21042,@nadeowinataa yessss semangat bro
21043,"Struick line up mulu anj, dari pertama main di..."


In [42]:
df.dropna(inplace=True)
df.isnull().sum()

,0
text,2811


In [ ]:
slang_df = pd.read_csv("https://raw.githubusercontent.com/MIAbidin/Sentimen-Timnas-App/refs/heads/main/Notebook/dataset/slang_dict.csv")
slang_dict = dict(zip(slang_df['tidak_baku'], slang_df['kata_baku']))

import re
import emoji

emoji_mapping = {
    ":loudly_crying_face:": "sangat sedih",
    ":crying_face:": "sedih",
    ":rolling_on_the_floor_laughing:": "tertawa",
    ":flexed_biceps:": "kuat",
    ":eagle:": "elang",
    ":folded_hands:": "berdoa",
    ":palms_up_together:": "memohon",
    ":thumbs_up:": "bagus",
    ":white_heart:": "cinta",
    ":red_heart:": "cinta"
}


def convert_emojis_to_words(text):
    return emoji.replace_emoji(
        text,
        replace=lambda emoji_char, _: emoji_mapping.get(emoji.demojize(emoji_char), '') + ' '
    ).strip()



def clean_text(text):
    text = str(text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'[_#]', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = convert_emojis_to_words(text)  # Konversi emoji ke kata biasa
    text = re.sub(r'[^\w\s]', '', text)  # Hilangkan tanda baca setelah konversi
    text = re.sub(r'\s+', ' ', text)     # Normalisasi spasi ganda
    text = text.strip()
    return text


# Text processing pipeline functions
def casefold_text(text):
    return text.lower()

def fix_slangwords(text):
    words = text.split()
    return ' '.join([slang_dict.get(word.lower(), word) for word in words])

def tokenizing_text(text):
    return word_tokenize(text)

def remove_stopwords(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update([ "yah", "abal", "yt", "yth", "yu", "yuk", "ml", 'iya', 'yaa', 'gak', 'nya', 'na', 'sih', 'ku', "di", "ga", "ya", "gaa", "loh", "kah", "woi", "woii", "woy", "bro", "dr", "mas", "wkwk", "aja", "bang", "si", "lu", "yg"])
    return [txt for txt in text if txt not in listStopwords]

# Buat stemmer satu kali saja
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming versi cepat
def stemming_text(tokens):
    return [stemmer.stem(word) for word in tokens]



In [ ]:
df['text_clean'] = df['text'].progress_apply(clean_text)
df['text_casefolded'] = df['text_clean'].progress_apply(casefold_text)
df['text_slangfixed'] = df['text_casefolded'].progress_apply(fix_slangwords)
df['text_tokenized'] = df['text_slangfixed'].progress_apply(tokenizing_text)
df['text_filtered'] = df['text_tokenized'].progress_apply(remove_stopwords)
df['text_stemmed'] = df['text_filtered'].progress_apply(stemming_text)
df['text_final'] = df['text_stemmed'].progress_apply(lambda x: ' '.join(x))

100%|██████████| 21023/21023 [00:00<00:00, 745379.07it/s]


In [ ]:
df

,id_file,name_file,created_at,username,text,sentiment,text_clean,text_casefolded,text_slangfixed,text_tokenized,text_filtered,text_stemmed,text_final,lexicon_score,lexicon_sentiment
0,21.0,C2Se75epJj5,2024-01-19 16:37:56,arief_engrazefai,🔥🔥🔥,positive,,,,[],[],[],,0,neutral
1,38.0,C2o,2024-01-28 10:55:29,muhahsanitaqwim,🔥🔥🔥,neutral,,,,[],[],[],,0,neutral
2,21.0,C2Se75epJj5,2024-01-19 16:54:53,ugie07,Topp 🔥❤️,positive,Topp cinta,topp cinta,topp cinta,"[topp, cinta]","[topp, cinta]","[topp, cinta]",topp cinta,0,neutral
3,33.0,C2fBBTFrLwh,2024-01-24 13:35:05,neneng_andriyani,"Alhamdulillah... Tidak apa-apa, yang penting b...",positive,Alhamdulillah Tidak apaapa yang penting bisa m...,alhamdulillah tidak apaapa yang penting bisa m...,alhamdulillah tidak apaapa yang penting bisa m...,"[alhamdulillah, tidak, apaapa, yang, penting, ...","[alhamdulillah, apaapa, mencetak, gol, gawang,...","[alhamdulillah, apaapa, cetak, gol, gawang, je...",alhamdulillah apaapa mencetak gol gawang jepang,8,positive
4,38.0,C2o,2024-01-28 11:49:24,kampusturkicom,Doa terbaik untuk timnas Indonesia. Insya Alla...,positive,Doa terbaik untuk timnas Indonesia Insya Allah...,doa terbaik untuk timnas indonesia insya allah...,doa terbaik untuk timnas indonesia insya allah...,"[doa, terbaik, untuk, timnas, indonesia, insya...","[doa, terbaik, timnas, indonesia, insya, allah...","[doa, baik, timnas, indonesia, insya, allah, m...",doa terbaik timnas indonesia insya allah menan...,6,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21040,25.0,C2TtGhvu4Bm,2024-01-20 04:34:16,herysajaherysaja,Semoga lawan Jepang pemain tetap rendah diri.....,positive,Semoga lawan Jepang pemain tetap rendah diri A...,semoga lawan jepang pemain tetap rendah diri a...,semoga lawan jepang pemain tetap rendah diri a...,"[semoga, lawan, jepang, pemain, tetap, rendah,...","[semoga, lawan, jepang, pemain, rendah, ayo, b...","[moga, lawan, jepang, main, rendah, ayo, bukti]",semoga lawan jepang pemain rendah ayo buktikan,-6,negative
21041,4.0,C18P695JpH4,2024-01-11 02:16:40,gunturputrarizky,Gazzzz Garudaakkuuuu 🔥🔥🔥🙌🙌,positive,Gazzzz Garudaakkuuuu,gazzzz garudaakkuuuu,gazzzz garudaakkuuuu,"[gazzzz, garudaakkuuuu]","[gazzzz, garudaakkuuuu]","[gazzzz, garudaakkuuuu]",gazzzz garudaakkuuuu,0,neutral
21042,5.0,C2Ek9lfL7uE,2024-01-14 07:39:31,aditya0484,@nadeowinataa yessss semangat bro 🔥🔥🔥🔥,positive,yessss semangat bro,yessss semangat bro,yessss semangat bro,"[yessss, semangat, bro]","[yessss, semangat]","[yessss, semangat]",yessss semangat,3,positive
21043,9.0,C2H2QelL3Lt,2024-01-15 14:58:51,arya_________w,"Struick line up mulu anj, dari pertama main di...",negative,Struick line up mulu anj dari pertama main di ...,struick line up mulu anj dari pertama main di ...,rafael struick line up mulu anjing dari perta...,"[rafael, struick, line, up, mulu, anjing, dari...","[rafael, struick, line, mulu, anjing, bermain,...","[rafael, struick, line, mulu, anjing, main, ti...",rafael struick line mulu anjing bermain timnas...,1,positive


In [ ]:
df['text_clean'] = df['text'].progress_apply(clean_text)
df['text_casefolded'] = df['text_clean'].progress_apply(casefold_text)
df['text_slangfixed'] = df['text_casefolded'].progress_apply(fix_slangwords)
df['text_tokenized'] = df['text_slangfixed'].progress_apply(tokenizing_text)
df['text_filtered'] = df['text_tokenized'].progress_apply(remove_stopwords)
df['text_stemmed'] = df['text_filtered'].progress_apply(stemming_text)
df['text_final'] = df['text_stemmed'].progress_apply(lambda x: ' '.join(x))

100%|██████████| 21023/21023 [00:00<00:00, 745379.07it/s]


In [ ]:
df_select6 = df[['text_final','lexicon_sentiment']]
df_select6

,text_final,lexicon_sentiment
0,,neutral
1,,neutral
2,topp cinta,neutral
3,alhamdulillah apaapa mencetak gol gawang jepang,positive
4,doa terbaik timnas indonesia insya allah menan...,positive
...,...,...
21040,semoga lawan jepang pemain rendah ayo buktikan,negative
21041,gazzzz garudaakkuuuu,neutral
21042,yessss semangat,positive
21043,rafael struick line mulu anjing bermain timnas...,positive


In [ ]:
df_select6 = df_select6[df_select6['text_final'].str.strip() != '']

In [ ]:
df_select6

,text_final,lexicon_sentiment
2,topp cinta,neutral
3,alhamdulillah apaapa mencetak gol gawang jepang,positive
4,doa terbaik timnas indonesia insya allah menan...,positive
6,gol membawa timnas lolos babak bahrain besok k...,negative
7,melia kitagaruda elang,positive
...,...,...
21039,cinta cinta,neutral
21040,semoga lawan jepang pemain rendah ayo buktikan,negative
21041,gazzzz garudaakkuuuu,neutral
21042,yessss semangat,positive


In [ ]:
import csv
import requests
from io import StringIO

def load_lexicon(url):
    lexicon = {}
    response = requests.get(url)
    if response.status_code == 200:
        reader = csv.reader(StringIO(response.text), delimiter='\t')  # ganti delimiter jadi tab (\t)
        next(reader)  # Lewati header
        for row in reader:
            if len(row) >= 2:
                lexicon[row[0]] = int(row[1])
    else:
        print(f"Failed to fetch lexicon from {url}")
    return lexicon



# Load lexicon positif dan negatif
lexicon_positive = load_lexicon('https://raw.githubusercontent.com/MIAbidin/Sentimen-Timnas-App/refs/heads/main/Notebook/dataset/positive.tsv')
lexicon_negative = load_lexicon('https://raw.githubusercontent.com/MIAbidin/Sentimen-Timnas-App/refs/heads/main/Notebook/dataset/negative.tsv')


In [ ]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        score += lexicon_positive.get(word, 0)
    for word in text:
        score += lexicon_negative.get(word, 0)

    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity


In [ ]:
df['text_clean'] = df['text'].progress_apply(clean_text)
df['text_casefolded'] = df['text_clean'].progress_apply(casefold_text)
df['text_slangfixed'] = df['text_casefolded'].progress_apply(fix_slangwords)
df['text_tokenized'] = df['text_slangfixed'].progress_apply(tokenizing_text)
df['text_filtered'] = df['text_tokenized'].progress_apply(remove_stopwords)
df['text_stemmed'] = df['text_filtered'].progress_apply(stemming_text)
df['text_final'] = df['text_stemmed'].progress_apply(lambda x: ' '.join(x))

100%|██████████| 21023/21023 [00:00<00:00, 745379.07it/s]


In [ ]:
df_select6 = df[['text_final','lexicon_sentiment']]
df_select6

,text_final,lexicon_sentiment
0,,neutral
1,,neutral
2,topp cinta,neutral
3,alhamdulillah apaapa mencetak gol gawang jepang,positive
4,doa terbaik timnas indonesia insya allah menan...,positive
...,...,...
21040,semoga lawan jepang pemain rendah ayo buktikan,negative
21041,gazzzz garudaakkuuuu,neutral
21042,yessss semangat,positive
21043,rafael struick line mulu anjing bermain timnas...,positive


In [ ]:
df_select6 = df_select6[df_select6['text_final'].str.strip() != '']

In [ ]:
df_select6

,text_final,lexicon_sentiment
2,topp cinta,neutral
3,alhamdulillah apaapa mencetak gol gawang jepang,positive
4,doa terbaik timnas indonesia insya allah menan...,positive
6,gol membawa timnas lolos babak bahrain besok k...,negative
7,melia kitagaruda elang,positive
...,...,...
21039,cinta cinta,neutral
21040,semoga lawan jepang pemain rendah ayo buktikan,negative
21041,gazzzz garudaakkuuuu,neutral
21042,yessss semangat,positive


In [ ]:
import csv
import requests
from io import StringIO

def load_lexicon(url):
    lexicon = {}
    response = requests.get(url)
    if response.status_code == 200:
        reader = csv.reader(StringIO(response.text), delimiter='\t')  # ganti delimiter jadi tab (\t)
        next(reader)  # Lewati header
        for row in reader:
            if len(row) >= 2:
                lexicon[row[0]] = int(row[1])
    else:
        print(f"Failed to fetch lexicon from {url}")
    return lexicon



# Load lexicon positif dan negatif
lexicon_positive = load_lexicon('https://raw.githubusercontent.com/MIAbidin/Sentimen-Timnas-App/refs/heads/main/Notebook/dataset/positive.tsv')
lexicon_negative = load_lexicon('https://raw.githubusercontent.com/MIAbidin/Sentimen-Timnas-App/refs/heads/main/Notebook/dataset/negative.tsv')


In [ ]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        score += lexicon_positive.get(word, 0)
    for word in text:
        score += lexicon_negative.get(word, 0)

    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity


In [ ]:
# Terapkan ke kolom text_filtered (token hasil stopword removal)
df[['lexicon_score', 'lexicon_sentiment']] = df['text_stemmed'].progress_apply(
    lambda x: pd.Series(sentiment_analysis_lexicon_indonesia(x))
)


100%|██████████| 21023/21023 [00:03<00:00, 5348.74it/s]


In [ ]:
train_val_df, test_df = train_test_split(
    df_select,
    test_size=0.2,
    random_state=42,
    stratify=df_select['lexicon_sentiment']
)
train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.2,
    random_state=42,
    stratify=train_val_df['lexicon_sentiment']
)

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)


Train shape: (11410, 2)
Validation shape: (2853, 2)
Test shape: (3566, 2)


In [ ]:
train_df.to_csv("train.tsv", sep="\t", index=False, header=False)
val_df.to_csv("valid.tsv", sep="\t", index=False, header=False)
test_df.to_csv("test.tsv", sep="\t", index=False, header=False)